In [1]:
# Bu sefer  fine-tuning için herhangi bir parametreyi değiştirmeyeceğiz.
 # Model bildiği hiç bir şeyi değiştirmeyecek
  # Bloom modelinden bir modeli alıp 2 farklı veri kümesiyle eğiteceğiz.
   # 1 tanesi promptlar oluşturmak için diğer nefret cümlelerini tespiti için.

In [2]:
# ilgili kütüphanler

!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q accelerate==0.29.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.4 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
# yine küçük bir model kullanıyoruz. Eğer iyi bir GPU varsa daha büyük model de denenebilir.

model_name = "bigscience/bloomz-560m"

NUM_VIRTUAL_TOKENS = 8
NUM_EPOCHS_PROMPT = 50
NUM_EPOCHS_CLASSIFIER = 50
device = "cuda"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map=device
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [6]:
# elimizdeki modeli fine_tuning etmeden deneyelim. Bizim için
# bu görevi yapacak bir fonksiyon

def get_output(model,inputs,max_new_tokens=100):
  outputs = model.generate(
      input_ids = inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      max_new_tokens = max_new_tokens,
      repetition_penalty = 1.5,
      early_stopping=True,
      eos_token_id = tokenizer.eos_token_id
  )

  return outputs

In [9]:
# Fine tune ettiğimiz model ile önceki modeli kıyaslamak için iki modelden de aynı girdiyi verip sonuç bekleyeceğiz.
 # prompt oluşturan bir model kuruyoruz.fitness koçu gibi davranması için bir prompt çıktısı vermesini bekliyorum.

input_prompt = tokenizer("Act as a fitness trainer. Prompt: ", return_tensors="pt")
foundational_outputs_prompt = get_output(
     foundational_model,
     input_prompt.to(device),
     max_new_tokens=50
 )

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['Act as a fitness trainer. Prompt:  Follow up with your clients']


In [ ]:
# Görüldüğü gibi model görevi tam anlamıyor. istediğimiz çıktıyı vermedi.

In [11]:
# Prompt üretmek için ilgili veri seti

import os
from datasets import load_dataset

dataset_prompt = "fka/awesome-chatgpt-prompts"


In [12]:
# Almak istediğimiz sonuca göre veriyi evriltiyoruz.
 # Aşağıdaki fonksiyonla veriyi kendi amacımıza uygun hale getiriyoruz.


def concatenate_columns_prompt(dataset):
  def concatenate(example):
    example["prompt"] = "Act as a {}. prompt: {}". format(example["act"], example["prompt"])
    return example
  dataset = dataset.map(concatenate)
  return dataset

In [16]:
# veriyi yükleme
data_prompt = load_dataset(dataset_prompt)

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
data_prompt

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})

In [20]:
# şimdi aşağıdakini yukardaki fonksiyona sokarak şunu yapmış olacağız;
 # örneğin; act as a Linux terminal. Prompt: I want you to act as a linu....  gibi bir eğitim verimiz olacak elimizde.
  # çünkü elde etmek istediğimiz model bun davranışı sergilesin istiyoruz.
print(data_prompt["train"][:1])

{'act': ['Linux Terminal'], 'prompt': ['I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd']}


In [21]:
data_prompt["train"] = concatenate_columns_prompt(data_prompt["train"])
print(data_prompt["train"][:1])

# elde ettiğimiz sonuç prompt sütununa yazıldı yani bu sütun yeniden fonksiyon ile yaniden düzenlendi.
 # artık act sütununa ihtiyacımız yok.

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

{'act': ['Linux Terminal'], 'prompt': ['Act as a Linux Terminal. prompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd']}


In [22]:
data_prompt = data_prompt.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
train_sample_prompt = data_prompt["train"].remove_columns("act")

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

In [25]:
print(train_sample_prompt[:1])

 # Eğitim verimiz nihayet hazır.
  # ilgili satırlar, tokenize edilmiş veriler, ve attentions mask....

{'prompt': ['Act as a Linux Terminal. prompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd'], 'input_ids': [[8972, 661, 267, 36647, 68320, 17, 39841, 29, 473, 4026, 1152, 427, 1769, 661, 267, 104105, 28434, 17, 473, 2152, 4105, 49123, 530, 1152, 2152, 57502, 1002, 3595, 368, 28434, 3403, 6460, 17, 473, 4026, 1152, 427, 3804, 57502, 1002, 368, 28434, 10014, 14652, 2592, 19826, 4400, 10973, 15, 530, 16915, 4384, 17, 727, 1130, 11602, 184637, 17, 727, 1130, 4105, 49123, 35262, 473, 32247, 1152, 427, 727, 1427, 17, 3262, 707, 3423, 427, 13485, 1152, 7747, 361, 170205, 15, 707, 2152, 727, 1427, 1331, 55385, 5484, 14

In [27]:
# Prompt Fine Tuning görevi için ilgili kütüphaneler

from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generation_config_prompt = PromptTuningConfig(
    task_type = TaskType.CAUSAL_LM, # text üretim görevini temsil ediyor
    prompt_tuning_init = PromptTuningInit.RANDOM ,
    num_virtual_tokens = NUM_VIRTUAL_TOKENS, # eklenecek sanal tokenlar. model eğitirken kullanılan bir teknik
    tokenizer_name_or_path = model_name # önceden eğitilmiş bizim seçtiğimiz model
)

In [28]:
#görülen o ki modelin gerçek parametrelerinden sadece 1000 de 1 ini eğiteceğiz.

peft_model_prompt = get_peft_model(foundational_model, generation_config_prompt)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [32]:
# training argumanlarını oluşturuyorz. bu argümanları fine tune edeceğimiz iki model için de kullanacağız.

from transformers import TrainingArguments

def create_training_arguments(path, learning_rate =0.003, epochs=5, autobatch=True):
  training_args = TrainingArguments(
      output_dir = path,
      auto_find_batch_size=autobatch,
      learning_rate=learning_rate,
      num_train_epochs=epochs


  )
  return training_args

In [31]:
import os

working_dir = "./"

# eğiteceğimiz prompt modeli ve sınıflandırıcı modeli ayrı dosyalara kaydetmek mantıklı

output_dir_prompt = os.path.join(working_dir, "peft_outputs_prompt")
output_dir_classifier = os.path.join(working_dir, "peft_outputs_classifier")


In [33]:
# ilk modelin eğitimi

# eğitim nesnesi

training_args_prompt = create_training_arguments(output_dir_prompt,
                                                 3e-2,
                                                 NUM_EPOCHS_PROMPT)

In [41]:
from transformers import Trainer, DataCollatorForLanguageModeling

#  Eğitim fonksiyonu

def create_trainer(model, training_args, train_dataset):
  trainer= Trainer(
      model = model,
      args= training_args,
      train_dataset = train_dataset,
      data_collator = DataCollatorForLanguageModeling(tokenizer, mlm =False)
  )
  return trainer

In [42]:
# Prompt geliştirmek için eğitiyor olduğumuz model

trainer_prompt= create_trainer(peft_model_prompt,
               training_args_prompt,
              train_sample_prompt)

trainer_prompt.train()

Step,Training Loss


Step,Training Loss
500,2.741300
1000,2.560600
1500,2.499200


TrainOutput(global_step=1950, training_loss=2.5715753330328526, metrics={'train_runtime': 955.5282, 'train_samples_per_second': 8.006, 'train_steps_per_second': 2.041, 'total_flos': 1912008157249536.0, 'train_loss': 2.5715753330328526, 'epoch': 50.0})

In [ ]:
"""
   Toparlayalım-
1- Tokenizer ve dil modeli nesnesi oluşturuldu

2- Eğitimsiz modelin performansı görüldü

3- Veri seti yüklendi ve amacımız doğrultusunda düzenlendi

4- Prompt tuning işlemi için prompttuningconfig nesnesi tanımlandı ( modelin proptları daha etkili kullanması için)

5- peft model nesnesi tanımlandı

6- trainingargument tanımlandı ( model parametrelerinin belirlenmesi için)

   * peft model için dil modeli ve config model
   * Train için peft model, training args, ve veri seti kullanıldı


peft_model = peft( dil_modeli, train_config)
trainer(peft_model, training_args, veri_Seti).train()

"""

In [ ]:
# modeli içe aktarma

trainer_prompt.model.save_pretrained(output_dir_prompt)

In [ ]:
from peft  import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                              output_dir_prompt,
                                              is_trainable=False)

loaded_model_prompt_outputs = get_outputs(
    loaded_model_peft,
    input_prompt,
    max_new_tokens=50,
)

print(tokenizer.batch_decode(loaded_model_prompt_outputs,skip_special_tokens=True))

In [ ]:
# çok daha iyi bir modelle daha ubüyük parametrelerle daha iyi sonuçlar verecektir.

In [ ]:
########################################################################
#### ## ## ## ### ## ###### # #### #### ##### #### ### # ##### ### #####
########################################################################

In [48]:
## NEfret cümleelrini tespit etmek için

input_classifier = tokenizer("Sentence: Head is the shape of a light bulb: Label: " , return_tensors="pt")
foundational_outputs_classifier = get_output(foundational_model,
                                           input_classifier.to(device),
                                           max_new_tokens = 50)
print(tokenizer.batch_decode(foundational_outputs_classifier, skip_special_tokens=True))

['Sentence: Head is the shape of a light bulb: Label:  head']


In [ ]:
# görüldüğü gibi model ne yapacağını bilemiyor.

In [49]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [55]:
data_classifier = load_dataset(dataset_classifier)
data_classifier["train"] = concatenate_columns_classifier(data_classifier["train"])
data_classifier = data_classifier.map(lambda samples: tokenizer(samples["text"]), batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(["label", "label_text","text"])

In [ ]:
# modelin ihtiyacı olmayan tüm sütunları kaldırdım. tokenize edilmiş sütun ve attentonmask yeterli.

In [56]:
###   tuning config

# prompt tuninginit.text yaptık bu sayede prompt tuning init text objesini amacımız dğrultusunda metin ile yönlendirebilriz.

config = PromptTuningConfig(
    task_type = TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Indicates whether the sentence contains hate speech or not",
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name
)

In [59]:
peft_model_classifier = get_peft_model(foundational_model, config)
print(peft_model_classifier.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [62]:
#daha önce hazırladığımız training parametreleri kullanabiliriz.

training_args_classifier = create_training_arguments(output_dir_classifier,
                                                    3e-2,
                                                    NUM_EPOCHS_CLASSIFIER)

In [ ]:
# Nefret söylemi tespiti yapan modelin eğitimi

trainer_classifier = create_trainer(peft_model_classifier,
                                    training_args_classifier,
                                    train_sample_classifier)
trainer_classifier.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss
500,3.285500
1000,3.228400
1500,3.228700
2000,3.218000
2500,3.207300
3000,3.212500
3500,3.173300
4000,3.175600
4500,3.210500
5000,3.160300


In [ ]:
# modeli içie aktra

trainer_classifier.model.save_pretrained(output_dir_classifier)

# dışa aktar

loaded_model_peft.load_adapter(output_dir_classifier,adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [ ]:
loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier, max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

In [ ]:
# Prompt tuning metodu çok iyi sonuç veriyor. Bu şekilde model görevini daha iyi kavrıyor.

# bu yöntemle modelin herhangi bir ağırlığı değiştiriilmez.